Для того чтобы видоизменить операцию
подключения модулей, можно получить данные
о их названиях, перекрыв метод visit_Import, а
затем, добавить их в условном операторе, проверяющем наличие точки выхода.

вот это мне не совсем понятно.

Наибольшее время уходит на генерацию кода на основе АСД, это объясняется тем, что модуль codegen является сторонней разработкой
на чистом Python, в то время как модуль ast –
внутренний с оптимизацией на Си. Вместо
формирования кода из АСД с помощью codegen,
можно использовать непосредственную компиляцию через метод compile:
s = compile(tree, filename="<ast>",
 mode="eval")
либо при задаче непосредственного выполнения:
exec(compile(tree, filename="<ast>"
 mode="exec")) 

In [19]:
import sys, getopt
import codegen
import ast
import showast
import ctree
import random
import astpretty
import hashlib
import codecs
from random_word import RandomWords
from cryptography.fernet import Fernet

r = RandomWords()

# were installed

# pip3 install graphviz
# pip3 install codegen
# sudo apt-get install graphviz
# pip3 install ctree
# pip3 install showast
# pip3 install astpretty
# pip install uncompyle6

In [2]:
def exec_tree(tree):
    print()
    print("=== AST ===")
    print(ast.dump(tree))
    print('---')
    
    astpretty.pprint(tree, show_offsets=False)
    print('---')

    tree_fixed = ast.fix_missing_locations(tree)
#     tree_fixed = tree
    code_obj = compile(tree_fixed, '<string>', 'exec')
    
    exec(code_obj)

# Усложенение констант

In [3]:
class ObfuscatorConstant(ast.NodeTransformer):
    def visit_Constant(self, node):
        if isinstance(node, ast.Num):
            value = node.n
            value_left = value // 2
            value_right = value - value_left
            return ast.BinOp(left=ast.Constant(value=value_left, kind=None),
                             op=ast.Add(), right=ast.Constant(value=value_right, kind=None))
        return node

path_middle_source = './obfuscated/examples/ast/elementary_source.py'
f = open(path_middle_source, 'r', encoding="utf-8")
tree = ast.parse(f.read(), mode="exec")
# show_source(f.read())
f.close()

exec_tree(tree)

## apply the transformations
obf = ObfuscatorConstant()
new_tree = obf.visit(tree)
exec_tree(new_tree)

# print(codegen.to_source(tree2))
# open(path, 'w').write(result)


=== AST ===
Module(body=[Assign(targets=[Name(id='a', ctx=Store())], value=BinOp(left=BinOp(left=Constant(value=1, kind=None), op=Add(), right=Constant(value=2, kind=None)), op=Add(), right=Constant(value=3, kind=None)), type_comment=None), Assign(targets=[Name(id='b', ctx=Store())], value=BinOp(left=BinOp(left=Constant(value='1', kind=None), op=Add(), right=Constant(value='2', kind=None)), op=Add(), right=Constant(value='3', kind=None)), type_comment=None)], type_ignores=[])
---
Module(
    body=[
        Assign(
            targets=[Name(id='a', ctx=Store())],
            value=BinOp(
                left=BinOp(
                    left=Constant(value=1, kind=None),
                    op=Add(),
                    right=Constant(value=2, kind=None),
                ),
                op=Add(),
                right=Constant(value=3, kind=None),
            ),
            type_comment=None,
        ),
        Assign(
            targets=[Name(id='b', ctx=Store())],
            value

# Усложнение констант - деобфускатор

Для простых алгоритмов преобразования не
сложно разработать алгоритм деобфускации,
использующий то же самое АСД. Например, можно получить результат сложения строк или
чисел, перекрыв оператор BinOp

Мб я перекрыла все операции со строками и числами, что в принципе довольно удобно

In [4]:
class DeobfuscatorConstant(ast.NodeTransformer):
    def visit_BinOp(self, node):
        node = self.generic_visit(node)
        if isinstance(node.left, ast.Num) and isinstance(node.right, ast.Num):
            return self.do_eval(node, ast.Num)
        if isinstance(node.left, ast.Str) and isinstance(node.right, ast.Str):
            return self.do_eval(node, ast.Str)
        return node
    def do_eval(self, node, n_type):
        code = codegen.to_source(node)
        b = ast.parse(code, mode='eval')
        code_eval = compile(b, '', 'eval')
        evaluated = eval(code_eval)
        new_node = ast.parse(str(evaluated), mode='eval')
        if n_type == ast.Str:
            new_node.body.value = str(new_node.body.value)
        return new_node.body


path_elementary_source = './obfuscated/examples/ast/elementary_source.py'
f = open(path_elementary_source, 'r', encoding="utf-8")
tree = ast.parse(f.read(), mode="exec")
# show_source(f.read())
f.close()

exec_tree(tree)

## apply the transformations
deobf = DeobfuscatorConstant()

new_tree = deobf.visit(tree)
exec_tree(new_tree)

print(codegen.to_source(new_tree))


=== AST ===
Module(body=[Assign(targets=[Name(id='a', ctx=Store())], value=BinOp(left=BinOp(left=Constant(value=1, kind=None), op=Add(), right=Constant(value=2, kind=None)), op=Add(), right=Constant(value=3, kind=None)), type_comment=None), Assign(targets=[Name(id='b', ctx=Store())], value=BinOp(left=BinOp(left=Constant(value='1', kind=None), op=Add(), right=Constant(value='2', kind=None)), op=Add(), right=Constant(value='3', kind=None)), type_comment=None)], type_ignores=[])
---
Module(
    body=[
        Assign(
            targets=[Name(id='a', ctx=Store())],
            value=BinOp(
                left=BinOp(
                    left=Constant(value=1, kind=None),
                    op=Add(),
                    right=Constant(value=2, kind=None),
                ),
                op=Add(),
                right=Constant(value=3, kind=None),
            ),
            type_comment=None,
        ),
        Assign(
            targets=[Name(id='b', ctx=Store())],
            value

# Кодирование строковых значений

String операции - переворачивание и конкатенация слабоваты для обфускации, человеческим глазом очень просто увидеть, что там написано. Надо дополнять сменой кодировок.

В принципе, если применить это на все строковые данные + какое-нибудь шифрование, смена кодировок и прочее - можно занять человеческому деобфускатору минут 20 ручного подбора, может быть отсеятся ленивые.

In [16]:
class ObfuscatorString(ast.NodeTransformer):
    def visit_Str(self, node):
        def rev(x):
            print(x)
            print('x in other side '+x[::-1])
            return ast.Subscript(value=ast.Str(s=x[::-1], kind=None), 
                             slice=ast.Slice(lower=None, upper=None, step=ast.Constant(n=-1, kind=None)), ctx=ast.Load())
        print('1 ' + node.s[:len(node.s)//2])
        print('2 ' + node.s[len(node.s)//2:])
        return ast.BinOp(left=rev(node.s[:len(node.s)//2]), op=ast.Add(), right=rev(node.s[len(node.s)//2:]))

path_compileall = './obfuscated/examples/compileall/source.py'
f = open(path_compileall, 'r', encoding="utf-8")
tree = ast.parse(f.read(), mode="exec")

# show_source(f.read())
f.close()

exec_tree(tree)

## apply the transformations
obf = ObfuscatorString()

new_tree = obf.visit(tree)
exec_tree(new_tree)

# print(codegen.to_source(tree2))
# open(path, 'w').write(result)


=== AST ===
Module(body=[FunctionDef(name='square_triangle_area', args=arguments(posonlyargs=[], args=[arg(arg='width', annotation=None, type_comment=None), arg(arg='height', annotation=None, type_comment=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=BinOp(left=Name(id='height', ctx=Load()), op=Mult(), right=Name(id='width', ctx=Load())), op=Div(), right=Constant(value=2, kind=None)))], decorator_list=[], returns=None, type_comment=None), Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Constant(value='Эта программа считает площадь прямоугольного треугольника.', kind=None)], keywords=[])), Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Constant(value='Введите значение длины: ', kind=None)], keywords=[])), Assign(targets=[Name(id='w', ctx=Store())], value=Call(func=Name(id='int', ctx=Load()), args=[Call(func=Name(id='input', ctx=Load()), args=[], keywords=[])], keywords=[]), type_comment=None), Expr(value=C

2
Введите значение высоты: 
2
Площадь прямоугольного треугольника = 2.0



In [22]:
class ObfuscatorStringEncoding(ast.NodeTransformer):
    def visit_Str(self, node):        
        byte_str = node.s.encode('utf-8')
        key = b'kV1IuPDgRc9fsYHNG3nifKSuMfm1fcJNeu8TUb7ouBU='
        encryption_type = Fernet(key)
        encrypted_message = encryption_type.encrypt(byte_str)
        encrypted_str = encrypted_message.decode('utf-8')
        print(encrypted_str)
#         print(Fernet('kV1IuPDgRc9fsYHNG3nifKSuMfm1fcJNeu8TUb7ouBU='.encode('utf-8')).decrypt(encrypted_str.encode('utf-8')).decode('utf-8'))
        return ast.Call(
                func=ast.Attribute(
                    value=ast.Call(
                        func=ast.Attribute(
                            value=ast.Call(
                                func=ast.Name(id='Fernet', ctx=ast.Load()),
                                args=[
                                    ast.Call(
                                        func=ast.Attribute(
                                            value=ast.Constant(value='kV1IuPDgRc9fsYHNG3nifKSuMfm1fcJNeu8TUb7ouBU=', kind=None),
                                            attr='encode',
                                            ctx=ast.Load()
                                        ),
                                        args=[ast.Constant(value='utf-8', kind=None)],
                                        keywords=[]
                                    )
                                ],
                                keywords=[]
                            ),
                            attr='decrypt',
                            ctx=ast.Load()
                        ),
                        args=[
                            ast.Call(
                                func=ast.Attribute(value=ast.Constant(value=encrypted_str, kind=None), attr='encode', ctx=ast.Load()),
                                args=[ast.Constant(value='utf-8', kind=None)],
                                keywords=[]
                            )
                        ],
                        keywords=[]
                    ),
                    attr='decode',
                    ctx=ast.Load()
                ),
                args=[ast.Constant(value='utf-8', kind=None)],
                keywords=[]
            )


path_compileall = './obfuscated/examples/compileall/source.py'
f = open(path_compileall, 'r', encoding="utf-8")
tree = ast.parse(f.read(), mode="exec")
# show_source(f.read())
f.close()

exec_tree(tree)

## apply the transformations
obf = ObfuscatorStringEncoding()

new_tree = obf.visit(tree)
exec_tree(new_tree)

# print(codegen.to_source(tree2))
# open(path, 'w').write(result)


=== AST ===
Module(body=[FunctionDef(name='square_triangle_area', args=arguments(posonlyargs=[], args=[arg(arg='width', annotation=None, type_comment=None), arg(arg='height', annotation=None, type_comment=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=BinOp(left=Name(id='height', ctx=Load()), op=Mult(), right=Name(id='width', ctx=Load())), op=Div(), right=Constant(value=2, kind=None)))], decorator_list=[], returns=None, type_comment=None), Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Constant(value='Эта программа считает площадь прямоугольного треугольника.', kind=None)], keywords=[])), Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Constant(value='Введите значение длины: ', kind=None)], keywords=[])), Assign(targets=[Name(id='w', ctx=Store())], value=Call(func=Name(id='int', ctx=Load()), args=[Call(func=Name(id='input', ctx=Load()), args=[], keywords=[])], keywords=[]), type_comment=None), Expr(value=C

2
Введите значение высоты: 
2
Площадь прямоугольного треугольника = 2.0



# Кодирование строковых значений - деобфускатор

In [7]:
class DeobfuscatorString(ast.NodeTransformer):
    def visit_Call(self, node):
        node = self.generic_visit(node)
        if isinstance(node.left, ast.Num) and isinstance(node.right, ast.Num):
            return self.do_eval(node, ast.Num)
        if isinstance(node.left, ast.Str) and isinstance(node.right, ast.Str):
            return self.do_eval(node, ast.Str)
        return node
    def is_input_inside(self, node):
        pass
    def do_eval(self, node):
        code = codegen.to_source(node)
        b = ast.parse(code, mode='eval')
        code_eval = compile(b, '', 'eval')
        evaluated = eval(code_eval)
        new_node = ast.parse(str(evaluated), mode='eval')
        
        return new_node.body


path_elementary_source = './obfuscated/examples/ast/elementary_source.py'
f = open(path_elementary_source, 'r', encoding="utf-8")
tree = ast.parse(f.read(), mode="exec")
# show_source(f.read())
f.close()

exec_tree(tree)

## apply the transformations
deobf = DeobfuscatorConstant()

new_tree = deobf.visit(tree)
exec_tree(new_tree)

print(codegen.to_source(new_tree))

IndentationError: expected an indented block (<ipython-input-7-bb91685db9c9>, line 11)

# Смена названий переменных, методов и классов

In [8]:
class NameObfuscatorAST(ast.NodeTransformer):
    def visit_ClassDef(self, node):
        if node.name[:2] != '__':
            node.name = self.obfuscate_class_name(node.name)
        for base in node.bases:
            if isinstance(base, ast.Name):
                if base.id in class_name_set:
                    base.id = class_name_dict[base.id]
        methods = [stmt for stmt in node.body if isinstance(stmt, ast.FunctionDef)]
        for method in methods:
            self.visit(method)
        return node
    def visit_Assign(self, node):
        for target in node.targets:
            if isinstance(target, ast.Name):
                if target.id[:2] != '__':
                    target.id = self.get_obfuscated_func_arg(target.id)
                    if target.id not in func_arg_set:
                        target.id = self.obfuscate_variable_name(target.id)
            elif isinstance(target, ast.Attribute) and isinstance(target.value, ast.Name):
                if target.value.id[:2] != '__' and target.value.id == 'self':
                    target.attr = self.get_obfuscated_func_arg(target.attr)
        self.visit(node.value)
        return node
    def get_obfuscated_func_arg(self, name):
        if name in func_arg_set:
            name = func_arg_dict[name]
        return name
    def visit_FunctionDef(self, node):
        if node.name[:2] != '__':
            node.name = self.obfuscate_function_name(node.name)
        for argname in node.args.args:
            if argname.arg != 'self':
                argname.arg = self.obfuscate_function_arg(argname.arg)
        node.body = [self.visit(x) for x in node.body]
        return node
    def visit_Name(self, node):
        node.id = self.get_obfuscated_name(node.id)
        return node
    def visit_Attribute(self, node):
        node.attr = self.get_obfuscated_name(node.attr)
        self.visit(node.value)
        return node
    def get_obfuscated_name(self, str_name):
        if str_name in func_name_set:
            str_name = func_name_dict[str_name]
        if str_name in func_arg_set:
            str_name = func_arg_dict[str_name]
        elif str_name in var_name_set:
            str_name = var_name_dict[str_name]
        elif str_name in class_name_set:
            str_name = class_name_dict[str_name]
        return str_name
    def obfuscate_class_name(self, name):
        class_name_set.add(name)
        class_name_dict[name] = self.obfuscate_string(name)
        return class_name_dict[name]
    def obfuscate_variable_name(self, name):
        var_name_set.add(name)
        var_name_dict[name] = self.obfuscate_string(name)
        return var_name_dict[name]
    def obfuscate_function_name(self, name):
        func_name_set.add(name)
        func_name_dict[name] = self.obfuscate_string(name)
        return func_name_dict[name]
    def obfuscate_function_arg(self, name):
        func_arg_set.add(name)
        func_arg_dict[name] = self.obfuscate_string(name)
        return func_arg_dict[name]
    def obfuscate_string(self, name):
        hash_object = hashlib.sha3_224(bytes(name, 'utf-8'))
        hex_dig = hash_object.hexdigest()
        return '_' + hex_dig

path_compileall = './obfuscated/examples/ast/complex_source.py'
f = open(path_compileall, 'r', encoding="utf-8")
tree = ast.parse(f.read(), mode="exec")
# show_source(f.read())
f.close()

exec_tree(tree)

## apply the transformations
obf = NameObfuscatorAST()

func_name_dict = dict()
func_name_set = set()
func_arg_dict = dict()
func_arg_set = set()
var_name_dict = dict()
var_name_set = set()
class_name_dict = dict()
class_name_set = set()

print('\n\n--- NOW NEW TREE ---\n\n')

new_tree = obf.visit(tree)
exec_tree(new_tree)

# print(codegen.to_source(tree2))
# open(path, 'w').write(result)


=== AST ===
Module(body=[ClassDef(name='Vehicle', bases=[Name(id='object', ctx=Load())], keywords=[], body=[Expr(value=Constant(value='\n    The Vehicle class\n    ', kind=None)), FunctionDef(name='__init__', args=arguments(posonlyargs=[], args=[arg(arg='self', annotation=None, type_comment=None), arg(arg='color', annotation=None, type_comment=None), arg(arg='doors', annotation=None, type_comment=None), arg(arg='tires', annotation=None, type_comment=None), arg(arg='vtype', annotation=None, type_comment=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Expr(value=Constant(value='Constructor', kind=None)), Assign(targets=[Attribute(value=Name(id='self', ctx=Load()), attr='color', ctx=Store())], value=Name(id='color', ctx=Load()), type_comment=None), Assign(targets=[Attribute(value=Name(id='self', ctx=Load()), attr='doors', ctx=Store())], value=Name(id='doors', ctx=Load()), type_comment=None), Assign(targets=[Attribute(value=Name(id='self', ctx=Load()), 

# Смена названий на что-то более менее приемлемое

In [11]:
class NameDeobfuscatorAST(ast.NodeTransformer):
    def __init__(self):
        self.name_dict = dict()
        self.name_set = set()
        ast.NodeTransformer.__init__(self)
    def visit_ClassDef(self, node):
        if node.name[:2] != '__':
            node.name = self.deobfuscate_name(node.name)
        for base in node.bases:
            if isinstance(base, ast.Name):
                if base.id in self.name_set:
                    base.id = self.name_dict[base.id]
        methods = [stmt for stmt in node.body if isinstance(stmt, ast.FunctionDef)]
        for method in methods:
            self.visit(method)
        return node
    def visit_Assign(self, node):
        for target in node.targets:
            if isinstance(target, ast.Name):
                if target.id[:2] != '__':
                    target.id = self.get_obfuscated_func_arg(target.id)
                    if target.id not in self.name_set:
                        target.id = self.deobfuscate_name(target.id)
            elif isinstance(target, ast.Attribute) and isinstance(target.value, ast.Name):
                if target.value.id[:2] != '__' and target.value.id == 'self':
                    target.attr = self.get_obfuscated_func_arg(target.attr)
        self.visit(node.value)
        return node
    def get_obfuscated_func_arg(self, name):
        if name in self.name_set:
            name = self.name_dict[name]
        return name
    def visit_FunctionDef(self, node):
        if node.name[:2] != '__':
            node.name = self.deobfuscate_name(node.name)
        for argname in node.args.args:
            if argname.arg != 'self':
                argname.arg = self.deobfuscate_name(argname.arg)
        node.body = [self.visit(x) for x in node.body]
        return node
    def visit_Name(self, node):
        if node.id in self.name_set:
            node.id = self.name_dict[node.id]
        if node.id in self.name_set:
            node.id = self.name_dict[node.id]
        elif node.id in self.name_set:
            node.id = self.name_dict[node.id]
        elif node.id in self.name_set:
            node.id = self.name_dict[node.id]
        return node
    def visit_Attribute(self, node):
        if node.attr in self.name_set:
            node.attr = self.name_dict[node.attr]
        if node.attr in self.name_set:
            node.attr = self.name_dict[node.attr]
        elif node.attr in self.name_set:
            node.attr = self.name_dict[node.attr]
        elif node.attr in self.name_set:
            node.attr = self.name_dict[node.attr]
        self.visit(node.value)
        return node
    def deobfuscate_name(self, name):
        if name not in self.name_set:
            self.name_set.add(name)
            self.name_dict[name] = self.random_name()
        return self.name_dict[name]
    def random_name(self):
        name = r.get_random_word(hasDictionaryDef="true", includePartOfSpeech="noun",
                                     minCorpusCount=9, maxCorpusCount=10,
                                     minDictionaryCount=1, maxDictionaryCount=10,
                                     minLength=3, maxLength=8)
        while name in self.name_set:
            name = r.get_random_word(hasDictionaryDef="true", includePartOfSpeech="noun",
                                     minCorpusCount=9, maxCorpusCount=10,
                                     minDictionaryCount=1, maxDictionaryCount=10,
                                     minLength=3, maxLength=8)
        print('new name is: ' + name)
        return name


path_compileall = './obfuscated/examples/ast/complex_source.py'
f = open(path_compileall, 'r', encoding="utf-8")
tree = ast.parse(f.read(), mode="exec")
# show_source(f.read())
f.close()

exec_tree(tree)

## apply the transformations
obf = NameDeobfuscatorAST()

print('\n\n--- NOW NEW TREE ---\n\n')

new_tree = obf.visit(tree)
exec_tree(new_tree)

# print(codegen.to_source(tree2))
# open(path, 'w').write(result)


=== AST ===
Module(body=[ClassDef(name='Vehicle', bases=[Name(id='object', ctx=Load())], keywords=[], body=[Expr(value=Constant(value='\n    The Vehicle class\n    ', kind=None)), FunctionDef(name='__init__', args=arguments(posonlyargs=[], args=[arg(arg='self', annotation=None, type_comment=None), arg(arg='color', annotation=None, type_comment=None), arg(arg='doors', annotation=None, type_comment=None), arg(arg='tires', annotation=None, type_comment=None), arg(arg='vtype', annotation=None, type_comment=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Expr(value=Constant(value='Constructor', kind=None)), Assign(targets=[Attribute(value=Name(id='self', ctx=Load()), attr='color', ctx=Store())], value=Name(id='color', ctx=Load()), type_comment=None), Assign(targets=[Attribute(value=Name(id='self', ctx=Load()), attr='doors', ctx=Store())], value=Name(id='doors', ctx=Load()), type_comment=None), Assign(targets=[Attribute(value=Name(id='self', ctx=Load()), 

new name is: fresnels
new name is: tenaille
new name is: resorter
new name is: ilicic
new name is: APIPA
new name is: antitop
new name is: gr.
new name is: sirenian
new name is: unshell
new name is: Goth.

=== AST ===
Module(body=[ClassDef(name='fresnels', bases=[Name(id='object', ctx=Load())], keywords=[], body=[Expr(value=Constant(value='\n    The Vehicle class\n    ', kind=None)), FunctionDef(name='__init__', args=arguments(posonlyargs=[], args=[arg(arg='self', annotation=None, type_comment=None), arg(arg='tenaille', annotation=None, type_comment=None), arg(arg='resorter', annotation=None, type_comment=None), arg(arg='ilicic', annotation=None, type_comment=None), arg(arg='APIPA', annotation=None, type_comment=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Expr(value=Constant(value='Constructor', kind=None)), Assign(targets=[Attribute(value=Name(id='self', ctx=Load()), attr='tenaille', ctx=Store())], value=Name(id='tenaille', ctx=Load()), type_com

# If условие

можно оставить простой метод, как тут, а можно сменить if условия на try except с известными исключениями

In [ ]:
class ObfuscatorAST(ast.NodeTransformer):
    def visit_If(self, node):
        node.test = ast.BoolOp(op=ast.And(),
                               values=[node.test,
                                       ast.Compare(left=ast.Constant(1, kind=None),
                                               ops=[ast.Gt()],
                                               comparators=[ast.Constant(0, kind=None)])])
        node.test = self.visit(node.test)
        return node

path_middle_source = './obfuscated/examples/ast/elementary_source.py'
f = open(path_middle_source, 'r', encoding="utf-8")
tree = ast.parse(f.read(), mode="exec")
# show_source(f.read())
f.close()

exec_tree(tree)

## apply the transformations
obf = ObfuscatorAST()

new_tree = obf.visit(tree)
exec_tree(new_tree)

# print(codegen.to_source(tree2))
# open(path, 'w').write(result)

# If - деобфускатор

вроде бы тут вообще даже достаточно не деобфусцировать, а просто показать граф потока управления - это как полуавтомат

# Тренировочное поле

тут просто черновик

In [ ]:
code = 'a = int(input())'
tree = ast.parse(code, mode="exec")

exec_tree(tree)
node = tree.body[0]
b = ast.parse(node, mode='eval')
astpretty.pprint(b)
a = ast.Expression(b)
code_eval = compile(a, '', 'eval')
evaluated = eval(code_eval)
new_node = ast.parse(str(evaluated), mode='eval')


In [ ]:
code = '''print(Fernet('kV1IuPDgRc9fsYHNG3nifKSuMfm1fcJNeu8TUb7ouBU='.encode('utf-8')).decrypt("gAAAAABgqBPVWlUpgDaVQqMAC0TaHtEyXFrVaf7XXyDAAqYT6rRq8qFIxEbOx_aw_nZGKj30kkrBuT06k_Ygi3jcQY3g1C60OMdijjx0_y7k0ZwzqyhSi2-HqkkOSmk45Nmpxgb0ieYbf2DpfEn3lwdc9OWHxXvYAA0tv2Y5K6r8zWVrP11S3NZNo2wdCJCacfr9fU4a_vokuI6wYOiIajhRxSvftcxMzQ==".encode('utf-8')).decode('utf-8'))'''
tree = ast.parse(code)
ast.dump(tree)
astpretty.pprint(tree, show_offsets=False)
exec_tree(tree)

In [ ]:
a = "Fernet('kV1IuPDgRc9fsYHNG3nifKSuMfm1fcJNeu8TUb7ouBU='.encode('utf-8')).decrypt(encrypted_str.encode('utf-8')).decode('utf-8')"
tree = ast.parse(a)
ast.dump(tree)
astpretty.pprint(tree, show_offsets=False)

## функции

In [ ]:
print(hashlib.algorithms_available)
print(hashlib.algorithms_guaranteed)

hash_object = hashlib.sha3_224(bytes('Hello World', 'utf-8'))
hex_dig = hash_object.hexdigest()
print(hex_dig)

In [ ]:
class ObfuscatorAST(ast.NodeTransformer):
    def visit_FunctionDef(self, node):
        node.name = self.obfuscate_function_name(node.name)
        return node
    def visit_Name(self, node):
        print(node.id)
        if node.id in func_name_set:
            node.id=func_name_dict[node.id]
        return node
    def obfuscate_function_name(self, name):
        hash_object = hashlib.sha3_224(bytes(name, 'utf-8'))
        hex_dig = hash_object.hexdigest()
        func_name_set.add(name)
        func_name_dict[name] = '_' + hex_dig
        print(name + ' ' + '_' + hex_dig)
        return '_' + hex_dig

path_compileall = './obfuscated/examples/compileall/source.py'
f = open(path_compileall, 'r', encoding="utf-8")
tree = ast.parse(f.read(), mode="exec")
# show_source(f.read())
f.close()

exec_tree(tree)

## apply the transformations
obf = ObfuscatorAST()

func_name_dict = dict()
func_name_set = set()
new_tree = obf.visit(tree)
exec_tree(new_tree)

# print(codegen.to_source(tree2))
# open(path, 'w').write(result)

In [ ]:
code = '''
a = 1 + 2 + 3
b = "1" + "2" + "3"
'''

tree = ast.parse(code)
tree_fixed = ast.fix_missing_locations(tree)
code_obj = compile(tree_fixed, '<string>', 'exec')
print('--- code_obj ---')
print(code_obj)
executed = exec(code_obj)
print('--- executed ---')
print(executed)

src = '[i**2 for i in range(10)]'
b = ast.parse(src, mode='eval')
code_eval = compile(b, '', 'eval')
print('--- eval code ---')
print(code_eval)
evaluated = eval(code_eval)
print('--- evaluated ---')
print(evaluated)
next_try = ast.parse(str(evaluated), mode='eval')
print(next_try)

print('--- now codegen 1 ---')
print(codegen.to_source(tree))
# print('--- now codegen 2 ---')
# print(codegen.to_source(code_obj))
# print('--- now codegen 3 ---')
# print(codegen.to_source(executed))
print('--- now codegen 4 ---')
print(codegen.to_source(next_try))

astpretty.pprint(b, show_offsets=False)
astpretty.pprint(next_try, show_offsets=False)

## Тестовая замена переменных

In [ ]:
class ConstantTransformer(ast.NodeTransformer):
    def visit_Constant(self, node):
        new_value = random.randint(-10, 10)
        print(f"replacing Constant value {node.value} -> {new_value} at lineno: {node.lineno}")
        new_node = ast.Constant(new_value, kind=None)
        return new_node

def exec_tree(tree):
    print()
    print("=== AST ===")
    print(ast.dump(tree))
    print('---')
    
    astpretty.pprint(tree, show_offsets=False)
    print('---')

    tree_fixed = ast.fix_missing_locations(tree)
    code_obj = compile(tree_fixed, '<string>', 'exec')
    
    exec(code_obj)

code = """
left_op = 1
right_op = 2
sum_two_things = left_op + right_op
other_sum = sum_two_things - 1

print(sum_two_things)
print(other_sum)
"""

tree = ast.parse(code)
exec_tree(tree)

## fix seed
random.seed(10)

## apply the transformations
vis = ConstantTransformer()
new_tree = vis.visit(tree)
exec_tree(new_tree)